In [1]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision.models as net
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os
import sys

In [2]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "D:\lyftauto\dataset"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./project_config.yaml")
print(cfg)

{'format_version': 4, 'model_params': {'model_architecture': 'resnet50', 'history_num_frames': 10, 'history_step_size': 1, 'history_delta_time': 0.1, 'future_num_frames': 50, 'future_step_size': 1, 'future_delta_time': 0.1}, 'raster_params': {'raster_size': [224, 224], 'pixel_size': [0.5, 0.5], 'ego_center': [0.25, 0.5], 'map_type': 'py_semantic', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_threshold': 0.5, 'disable_traffic_light_faces': False}, 'train_data_loader': {'key': 'scenes/train.zarr', 'batch_size': 12, 'shuffle': True, 'num_workers': 0}, 'val_data_loader': {'key': 'scenes/validate.zarr', 'batch_size': 12, 'shuffle': False, 'num_workers': 0}, 'test_data_loader': {'key': 'scenes/test.zarr', 'batch_size': 12, 'shuffle': False, 'num_workers': 0}, 'train_params': {'checkpoint_every_n_steps': 10000, 'max_num_epochs': 1, 'max_num_steps': 100, 'eval_every_n_steps': 10000}, 'val_p

In [3]:
def build_model(modeltype, cfg: Dict) -> torch.nn.Module:
    # load pre-trained Conv2D model
    model = None
    output_features = 0
    if modeltype == 0:
        model = net.resnet18(pretrained=True)
        output_features = 512
    elif modeltype == 1:
        model = net.resnet50(pretrained=True)
        output_features = 2048
    #elif modeltype == 2:
    #    model = net.alexnet(pretrained=True)
    #    output_features = 4096
    #elif modeltype == 3:
    #    model = net.vgg16(pretrained=True)
    #    output_features = 4096
    
    assert(model != None)
    assert(output_features != 0)

    # change input channels number to match the rasterizer's output
    num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
    num_in_channels = 3 + num_history_channels
    model.conv1 = nn.Conv2d(
        num_in_channels,
        model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=False,
    )
    # change output size to (X, Y) * number of future states
    num_targets = 2 * cfg["model_params"]["future_num_frames"]
    
    model.fc = nn.Linear(in_features=output_features, out_features=num_targets)

    return model

In [4]:
def forward(data, model, device, criterion):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    outputs = model(inputs).reshape(targets.shape)
    loss = criterion(outputs, targets)
    loss = loss * target_availabilities
    loss = loss.mean()
    return loss, outputs

In [5]:
train_cfg = cfg["train_data_loader"]

In [6]:

rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
print(train_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss(reduction="none")
cuda_status = torch.cuda.is_available()

In [8]:
model = [None]*4
for x in range(2):
    model[x] = build_model(x,cfg).to(device)
    if cuda_status:
        model[x] = model[x].cuda()
    
    optimizer = optim.Adam(model[x].parameters(), lr=1e-3)
    model[x].train()
    torch.set_grad_enabled(True)
    
    losslist = []

    progress_bar = tqdm(range(cfg["train_params"]["max_num_epochs"]))

    for _ in progress_bar:
        training_loss = 0
        for i, data in enumerate(train_dataloader):
            loss, _ = forward(data, model[x], device, criterion)
            training_loss += loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #progress_bar.set_description(f"loss: {loss.item()}")
            sys.stdout.write("\r" + str(i) + ": " + str(loss.item()))
            sys.stdout.flush()
            if i > cfg["train_params"]["max_num_steps"]:
                break

        training_loss /= i
        losslist.append(training_loss.item())
        progress_bar.set_description(f"loss: {training_loss.item()}")
    print("model_type: " + str(x) + " loss: " + str(sum(losslist)/len(losslist)))

  0%|          | 0/1 [00:00<?, ?it/s]

101: 8.471467018127441

loss: 41.13312911987305: 100%|██████████| 1/1 [01:12<00:00, 72.75s/it]


model_type: 0 loss: 41.13312911987305


  0%|          | 0/1 [00:00<?, ?it/s]

101: 37.986812591552734

loss: 31.257076263427734: 100%|██████████| 1/1 [01:24<00:00, 84.64s/it]

model_type: 1 loss: 31.257076263427734


In [9]:
num_frames_to_chop = 100
eval_cfg = cfg["val_data_loader"]
eval_base_path = create_chopped_dataset(dm.require(eval_cfg["key"]), cfg["raster_params"]["filter_agents_threshold"], 
                              num_frames_to_chop, cfg["model_params"]["future_num_frames"], MIN_FUTURE_STEPS)

E:\app\anaconda\lib\site-packages\l5kit\data\zarr_utils.py:243: RuntimeWarning: zarr dataset path doesn't exist. Open will fail for this dataset!
  output_dataset = ChunkedDataset(output_zarr)
copying:   4%|▍         | 669/16220 [01:35<39:51,  6.50it/s]

KeyboardInterrupt: 

In [ ]:
eval_zarr_path = str(Path(eval_base_path) / Path(dm.require(eval_cfg["key"])).name)
eval_mask_path = str(Path(eval_base_path) / "mask.npz")
eval_gt_path = str(Path(eval_base_path) / "gt.csv")

eval_zarr = ChunkedDataset(eval_zarr_path).open()
eval_mask = np.load(eval_mask_path)["arr_0"]
eval_dataset = AgentDataset(cfg, eval_zarr, rasterizer, agents_mask=eval_mask)
eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
                             num_workers=eval_cfg["num_workers"])
print(eval_dataset)

In [10]:
val_cfg = cfg["val_data_loader"]
val_zarr = ChunkedDataset(dm.require(val_cfg["key"])).open()
val_dataset = AgentDataset(cfg, val_zarr, rasterizer)
val_dataloader = DataLoader(val_dataset, shuffle=val_cfg["shuffle"], batch_size=val_cfg["batch_size"], 
                             num_workers=val_cfg["num_workers"])
print(val_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16220    |  4030296   | 312617887  |    29277930   |      111.97     |        248.48        |        77.57         |        24.85         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [11]:
for x in range(2):
    model[x].eval()
    torch.set_grad_enabled(False)
    if cuda_status:
        model[x] = model[x].cuda()

    val_loss = 0
    for i, data in enumerate(train_dataloader):
        loss, _ = forward(data, model[x], device, criterion)
        val_loss += loss.item()
        if i > cfg["val_params"]["max_num_steps"]:
            break
    
    print("model_type: " + str(x) + " err: " + str(val_loss))

    

model_type: 0 err: 2408.488511443138
model_type: 1 err: 731.5637989640236


AttributeError: 'NoneType' object has no attribute 'eval'